# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,597.647599,-0.841071,1234,596.806528,598.488670,1193.613056,1196.977341
6,555.463677,-0.099606,1234,555.364072,555.563283,1110.728143,1111.126566
7,926.777474,-0.994550,1234,925.782923,927.772024,1851.565847,1855.544048
8,45.366374,-0.225117,1234,45.141257,45.591491,90.282514,91.182982
9,791.863145,0.017353,1234,791.880498,791.845792,1583.760996,1583.691585
10,593.265716,-0.666417,1234,592.599299,593.932133,1185.198597,1187.864265
11,214.125419,0.721594,1234,214.847012,213.403825,429.694025,426.807650
12,11.554859,-0.988013,1234,10.566846,12.542872,21.133692,25.085744
13,392.944679,-0.732282,1234,392.212397,393.676961,784.424795,787.353922
14,606.165074,-0.077119,1234,606.087955,606.242193,1212.175911,1212.484386


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-596.806528,598.488670
6,-555.364072,555.563283
7,-925.782923,927.772024
8,-45.141257,45.591491
9,-791.880498,791.845792
10,-592.599299,593.932133
11,-214.847012,213.403825
12,-10.566846,12.542872
13,-392.212397,393.676961
14,-606.087955,606.242193


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-596.806528,598.488670
6,-555.364072,555.563283
7,-925.782923,927.772024
8,-45.141257,45.591491
9,-791.880498,791.845792
10,-592.599299,593.932133
11,-214.847012,213.403825
12,-10.566846,12.542872
13,-392.212397,393.676961
14,-606.087955,606.242193


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-596.80652823,  598.48867039],
       [-555.36407152,  555.56328301],
       [-925.78292327,  927.77202414],
       [ -45.14125713,   45.59149091],
       [-791.88049796,  791.84579244],
       [-592.59929859,  593.93213255],
       [-214.84701237,  213.40382479],
       [ -10.56684611,   12.54287191],
       [-392.21239748,  393.67696081],
       [-606.0879554 ,  606.24219276]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1191.613056,-1196.977341
6,1108.728143,-1111.126566
7,1849.565847,-1855.544048
8,88.282514,-91.182982
9,1581.760996,-1583.691585
10,1183.198597,-1187.864265
11,427.694025,-426.807650
12,19.133692,-25.085744
13,782.424795,-787.353922
14,1210.175911,-1212.484386


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.597648,0.079464
6,0.555464,0.450197
7,0.926777,0.002725
8,0.045366,0.387442
9,0.791863,0.508676
10,0.593266,0.166792
11,0.214125,0.860797
12,0.011555,0.005994
13,0.392945,0.133859
14,0.606165,0.461441


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,597.647599,-0.841071
6,555.463677,-0.099606
7,926.777474,-0.994550
8,45.366374,-0.225117
9,791.863145,0.017353
10,593.265716,-0.666417
11,214.125419,0.721594
12,11.554859,-0.988013
13,392.944679,-0.732282
14,606.165074,-0.077119


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
